In [44]:
import numpy as np
import scipy
import scipy.stats

import concurrent
import multiprocessing

In [62]:
# np.random.random() samples form a continuous uniform distribution between [0, 1)
# https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.random.html#numpy.random.random

def generate_random():
    return np.random.random()

def generate_random_seed():
    scipy.random.seed()
    return np.random.random()

In [27]:
def execute(exectutor, random_function, n_samples):
    random_numbers = []
    with exectutor(max_workers=multiprocessing.cpu_count()) as executor:
            future_to_random = {executor.submit(random_function): b for b in range(n_samples)}
            for future in concurrent.futures.as_completed(future_to_random):
                b = future_to_random[future]
                try:
                    random_numbers.append(future.result())
                except Exception as exc:
                    print(f"generated an exception: {exc}")
    return random_numbers

In [57]:
# ks test tests goodness of fit to uniform distribution.
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kstest.html
# Null hypothesis is that empirical distribution of x and 'uniform' equal
def test_uniform(x):
    return scipy.stats.kstest(x, 'uniform')

In [58]:
n_samples = 10

In [59]:
# ProcessPoolExecutor without seed
execute(concurrent.futures.ProcessPoolExecutor, generate_random, n_samples)

[0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083,
 0.3531318913277083]

That doesn't look random to me. Let's use the KS test to check.

In [61]:
x = execute(concurrent.futures.ProcessPoolExecutor, generate_random, 1000)
test_uniform(x).pvalue

7.0264984690686185e-12

Seems quite significant. We can reject the null. The two distributions are not the same with very high probability.

In [30]:
# Thread pool executor without seed
execute(concurrent.futures.ThreadPoolExecutor, generate_random, n_samples)

[0.4022700186390239,
 0.03626181650585103,
 0.8421808068397407,
 0.8373856895356356,
 0.5731373725691851,
 0.631598702196628,
 0.6504276922773542,
 0.5015282007961973,
 0.21039857328688172,
 0.581512328242947]

Looks better, let's make sure that it is.

In [63]:
x = execute(concurrent.futures.ThreadPoolExecutor, generate_random, 1000)
test_uniform(x).pvalue

0.7443606847918853

Another approach is:

In [31]:
# ProcessPoolExecutor with seed
execute(concurrent.futures.ProcessPoolExecutor, generate_random_seed, n_samples)

[0.15024702541952184,
 0.663013373426398,
 0.9071019315049326,
 0.5581711681531546,
 0.760395688185846,
 0.12881868459866508,
 0.6504844220709478,
 0.025152605613995127,
 0.5770563186769598,
 0.09618697683998012]

In [64]:
x = execute(concurrent.futures.ProcessPoolExecutor, generate_random_seed, 1000)
test_uniform(x).pvalue

0.6387485683349132